<h1>Task 2 Data Wrangling</h1>

In [34]:
import pandas as pd
import os
from nltk.corpus import stopwords
from nltk.util import ngrams
import string
import scipy
import numpy as np
import torch
import tensorflow as tf
import joblib
import xml.etree.ElementTree as ET
import re
from typing import List
from torch.utils.data import TensorDataset
from transformers import AutoTokenizer

pd.set_option('display.max_rows', 10)

<h2>MIMIC III</h2>

First we load the MIMIC III dataset, which will function as our training/development dataset. Before we can use the data for building our weak supervision models we still have to do some preprocessing of certain columns.

In [2]:
df_notes = pd.read_csv('./data/train/mimic-iii-clinical-database-1.4/NOTEEVENTS.csv', low_memory=False)

In [5]:
df_notes.shape

(2083180, 11)

In [5]:
df_notes.head()

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,174,22532,167853.0,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...
1,175,13702,107527.0,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-6-2**] Discharg...
2,176,13702,167118.0,2119-05-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2119-5-4**] D...
3,177,13702,196489.0,2124-08-18,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2124-7-21**] ...
4,178,26880,135453.0,2162-03-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2162-3-3**] D...


<h3>Preprocessing</h3>

In [3]:
notes_text = df_notes.TEXT

In [4]:
del df_notes # removing variables from memory for tidying up RAM usage

In [5]:
notes_text = notes_text.str.replace(r'\[\*\*(.*?)\*\*\]', '') # extract tag placeholders

/home/david/.local/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [6]:
notes_text = notes_text.str.replace(r'[0-9]+', '') # extract all digits

/home/david/.local/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [10]:
punctuation = string.punctuation.replace('/', '')
punctuation = punctuation.replace('\\', '') # excluding slashes from punctuation removal, since we need them to match with some disease

notes_text = notes_text.str.translate(str.maketrans('', '', punctuation))

In [12]:
stopwords = stopwords.words('english')
stopwords.remove('and')
stopwords.remove('or')

In [13]:
notes_text = notes_text.apply(lambda x: ' '.join([word for word in x.lower().split() if word not in (stopwords)]))

In [14]:
notes_text.to_csv('./data/train/notes_cleaned.csv', index=False)

In [15]:
del notes_text

In [30]:
notes_text = pd.read_csv('./data/train/notes_cleaned.csv', low_memory=False, chunksize = 10000, index_col = False)

For feeding the MIMIC III data to our labeling functions we opted for a n-gram approach - i. e. every note entry is split into x n-grams corresponding to x number of rows. Given the dimensions of our keyword lists we decided to generate every possible iteration from unigrams to 7-grams in order to then apply our labeling functions to every subset.

In [29]:
# processing ngrams in chunks for performance reasons

def get_ngrams(file_path, n, df):
    try:
        os.remove(file_path)
    except OSError:
        pass

    for subset in df:
        subset = subset.dropna()
        subset['ngrams'] = subset['TEXT'].str.split().apply(lambda x: list(map(' '.join, ngrams(x, n=n))))
        subset = (subset.assign(count=subset['ngrams'].str.len())
    .explode('ngrams')
    .query('count > 0'))
        subset['index_notes'] = subset.index
        subset = subset.drop(['count', 'TEXT'], axis=1)
        if not os.path.isfile(file_path):
            subset.to_csv(file_path, index=False)
        else:
            subset.to_csv(file_path, index=False, mode='a', header=False)
    return None

In [ ]:
get_ngrams('./data/train/ngrams/1grams.csv', 1, notes_text)

In [ ]:
get_ngrams('./data/train/ngrams/2grams.csv', 2, notes_text)

In [ ]:
get_ngrams('./data/train/ngrams/3grams.csv', 3, notes_text)

In [ ]:
get_ngrams('./data/train/ngrams/4grams.csv', 4, notes_text)

In [ ]:
get_ngrams('./data/train/ngrams/5grams.csv', 5, notes_text)

In [ ]:
get_ngrams('./data/train/ngrams/6grams.csv', 6, notes_text)

In [ ]:
get_ngrams('./data/train/ngrams/7grams.csv', 7, notes_text)

<h3>Labeling Functions</h3>

As a next step we load a text file containing a list of the most important clinical departments for our first labeling function. This list was scraped from different sources, which are also provided in the text file. The input from the text file is then transformed into a list, consisting of various ngrams, with each ngram denoting a clinical department. This list will then be applied to our ngrams from the MIMIC dataset where every exact match gets assigned the label "DEP".

In [4]:
with open("./data/keywords/departments_list.txt") as dataFile:
    departments = {}
    for line in dataFile:
        if line.strip() == '': # exclude unnecessary lines
            break
        else:
            line = line.split(':')
            key, value = line[0], line[1:]
            value = [i.replace(';', ',') for i in value]
            value = [i.split(',') for i in value]
            [[value]] = [value]
            value = [i.strip() for i in value]
            departments[key] = value

list(departments.items())[0:3]

[('Department of Admissions',
  ['Admission', 'Admissions', 'Admitting Department']),
 ('Department of Anaesthesia, Intensive Care Medicine and Pain Medicine',
  ['Anesthetics', 'Anesthesiology']),
 ('Department of Blood Group Serology and Transfusion Medicine',
  ['Serology', 'Transfusion Medicine'])]

In [5]:
departments_keywords = list(departments.values())
departments_keywords = [item.lower() for sublist in departments_keywords for item in sublist]

In [6]:
departments_keywords[0:3]

['admission', 'admissions', 'admitting department']

Another keywords list which we will use contains a collection of terms denoting clinical events - here every exact match in the MIMIC dataset gets assigned the label "EVE".

In [7]:
events_keywords_df = pd.read_csv('./data/keywords/Thesaurus.txt', delimiter='\t')

In [8]:
events_keywords_df = events_keywords_df.drop(events_keywords_df.columns[[0,1,2,5,6]], axis=1)
events_keywords_df = events_keywords_df.rename(columns={events_keywords_df.columns[0]:'Keywords', events_keywords_df.columns[1]:'Desc', events_keywords_df.columns[2]:'Label'})
events_keywords_df = events_keywords_df.drop(events_keywords_df[events_keywords_df.Label != 'Event'].index)

In [9]:
events_keywords = events_keywords_df.Keywords.str.replace(r'\|.+', '')
events_keywords = events_keywords.str.replace('/', ' ')

/home/david/.local/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [10]:
events_keywords = events_keywords.tolist()
events_keywords = [x.lower() for x in events_keywords]

In order to also extract evidence of problematic clinical events/occurences we create a list of negative phrases will then be matched to the label "PRO".

In [11]:
with open("./data/keywords/negCueWords.txt") as dataFile:
    problems_keywords = []
    for line in dataFile:
        res = line.split('|', 1)
        problems_keywords.append(res[0])

In [3]:
def ngrams_keywords(n, input_ls):
    """function for extracting subset of ngrams from a given keyword list"""
    output_ls = []
    for i in input_ls:
        ws = i.count(' ')
        if ws == n-1:
            output_ls.append(i)
    return output_ls

In [11]:
def labeling_function(file_path, df, keyword_list):
    """labeling function takes as input a dataframe and a list of keywords and encodes every ngram with 0/1 columns for all the possible labels (1 if exact match, else 0)"""
    try:
        os.remove(file_path)
    except OSError:
        pass

    for subset in df:
        subset = subset.dropna()
        for keyword in keyword_list:
            subset[keyword] = subset['ngrams'].map(lambda x: 1 if x == keyword  else 0)
    if not os.path.isfile(file_path):
        subset.to_csv(file_path, index=False)
    else:
        subset.to_csv(file_path, index=False, mode='a', header=False)
    return None

In [23]:
notes_ngrams = pd.read_csv('./data/train/ngrams/notes_1grams.csv', low_memory=False, chunksize = 500000, index_col = False)

In [12]:
departments_keywords_unigrams = ngrams_keywords(1, departments_keywords)
events_keywords_unigrams = ngrams_keywords(1, events_keywords)
problems_keywords_unigrams = ngrams_keywords(1, problems_keywords)
keywords_list = departments_keywords_unigrams + events_keywords_unigrams + problems_keywords_unigrams

In [9]:
labeling_function('./data/train/ngrams/notes_1grams_labeled.csv', notes_ngrams, keywords_list)

In [13]:
def explode_ngrams(file_path, df):
    """takes as input a dataframe of ngrams and creates for every word within a ngram a separate row, then the columns of the newly generated dataframe are also rearranged and renamed for clearer formatting"""
    try:
        os.remove(file_path)
    except OSError:
        pass

    for subset in df:
        subset['words'] = subset['ngrams'].str.split(' ')
        subset = subset.explode('words')

        cols = subset.columns.tolist()
        cols = cols[-1:] + cols[:-1]

        subset = subset[cols]

        subset = subset.rename(columns={'ngrams':'keywords'})

        if not os.path.isfile(file_path):
            subset.to_csv(file_path, index=False)
        else:
            subset.to_csv(file_path, index=False, mode='a', header=False)

<h2>i2b2 2012</h2>

<h3>Preprocessing</h3>

As our testing data we will use the i2b2 set from 2012 which consists of multiple clinical notes and annotations corresponding to different types of clinical events. Since all the annotated clinical notes are in .xml-format we first have to parse the separate files, clean them and finally concate the relevant attributes into a single dataframe. For these preprocessing steps we mostly adhere to the same pipeline we have already used for the MIMIC III data. 

In [2]:
punctuation = string.punctuation.replace('/', '')
punctuation = punctuation.replace('\\', '')

stopwords = stopwords.words('english')
stopwords.remove('and')
stopwords.remove('or')

In [6]:
ls_text = []
ls_note_id = []
ls_class = []

for filename in os.listdir('./data/test/2012-08-08.test-data.event-timex-groundtruth/xml/'):
    mytree = ET.parse('./data/test/2012-08-08.test-data.event-timex-groundtruth/xml/' + filename)
    myroot = mytree.getroot()
    for i in myroot[1]:
        if i.tag == 'EVENT': # filtering out time expressions, since they are not relevant for the given task
            if i.attrib.get('text').count(' ') == 0:
                ls_text.append(i.attrib.get('text'))
                ls_class.append(i.attrib.get('type'))
                ls_note_id.append(int(filename.replace('.xml', '')))

In [7]:
for filename in os.listdir('./data/test/2012-08-08.test-data.event-timex-groundtruth/xml/'):
    mytree = ET.parse('./data/test/2012-08-08.test-data.event-timex-groundtruth/xml/' + filename)
    myroot = mytree.getroot()
    t = myroot[0].text
    t = re.sub(r'[0-9]+', '', t)
    t = t.translate(str.maketrans('', '', punctuation))
    t = t.split(' ')
    t = [item.lower() for item in t if item]
    t = [item for item in t if item not in stopwords]
    for el in t:
        if el not in ls_text: # checking if words are already contained in the labeled subset, and if not append them to the existing arrays
            ls_text.append(el)
            ls_class.append('OTHER')
            ls_note_id.append(int(filename.replace('.xml', '')))

In [8]:
df_test = pd.DataFrame()
df_test['notes_id'] = ls_note_id
df_test['words'] = ls_text
df_test['classes'] = ls_class

df_test = df_test.dropna()

In [9]:
df_test.head()

,notes_id,words,classes
0,221,ADMISSION,OCCURRENCE
1,221,chemotherapy,TREATMENT
2,221,Cisplatin,TREATMENT
3,221,tolerated,OCCURRENCE
4,221,chemotherapy,TREATMENT


In [10]:
joblib.dump(df_test, './data/test/df_test.lib')

['./data/test/df_test.lib']

<h2>Knodle Matrices</h2>

Before we can finally put our generated data into knodle and build our models, we still have to create the necessary input matrices.

In [13]:
long_list = departments_keywords_unigrams + events_keywords_unigrams + problems_keywords_unigrams
df_T = pd.DataFrame(long_list)
df_T = df_T.rename(columns={df_T.columns[0]:'Keywords'})
df_T['DEP'] = df_T['Keywords'].map(lambda x: 1 if x in departments_keywords else 0)
df_T['EVE'] = df_T['Keywords'].map(lambda x: 1 if x in events_keywords else 0)
df_T['PRO'] = df_T['Keywords'].map(lambda x: 1 if x in problems_keywords else 0)

In [14]:
t_matrix = df_T.set_index('Keywords')
t_matrix = scipy.sparse.csr_matrix(t_matrix)
t_matrix = t_matrix.tocoo()

In [15]:
t_matrix = torch.sparse.LongTensor(torch.LongTensor([t_matrix.row.tolist(), t_matrix.col.tolist()]), torch.LongTensor(t_matrix.data.astype(np.int32)))

First, we create the T-matrix with the dimensions keywords x labels and save it to the joblib format for later use.

In [16]:
joblib.dump(t_matrix, './data/train/t_matrix.lib')

['./data/train/t_matrix.lib']

In [4]:
notes_1grams_matched = pd.read_csv('./data/train/ngrams/notes_1grams_labeled.csv', low_memory=False, chunksize=500000, index_col = False)

In [5]:
ls_matrix = []
for subset in notes_1grams_matched:
    try:
        subset = subset.set_index('ngrams')
        temp_matrix = scipy.sparse.csr_matrix(subset)
        ls_matrix.append(temp_matrix)
    except:
        pass

In [6]:
joblib.dump(ls_matrix, './data/train/ls_matrix.lib')

['./data/train/ls_matrix_scipy.lib']

In [2]:
ls_matrix = joblib.load('./data/train/ls_matrix.lib')

In [3]:
z_matrix = scipy.sparse.vstack(ls_matrix)

In [5]:
z_matrix = z_matrix.tocoo()
z_matrix = torch.sparse.LongTensor(torch.LongTensor([z_matrix.row.tolist(), z_matrix.col.tolist()]), torch.LongTensor(z_matrix.data.astype(np.int32)))

Secondly, we also create the Z-matrix which consists of our ngrams x keywords and an additional index column for the respective clinical note from which a given ngram stems.

In [6]:
joblib.dump(z_matrix, './data/train/z_matrix.lib')

['./data/train/z_matrix_conc_scipy.lib']

Finally, we also create the X-matrices and Y-matrix for our train and test data.

In [28]:
df_test = joblib.load('./data/test/df_test.lib')

In [2]:
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)


Here we utilise these two little helper functions - they make use of the transformers and the scipy package in order to create our X- and Y-matrices.

In [3]:
def convert_text_to_transformer_input(tokenizer, texts: List[str]) -> TensorDataset:
    encoding = tokenizer(texts, return_tensors="pt", padding=True, truncation=True)
    input_ids = encoding.get('input_ids')
    attention_mask = encoding.get('attention_mask')

    input_values_x = TensorDataset(input_ids, attention_mask)

    return input_values_x


def np_array_to_tensor_dataset(x: np.ndarray) -> TensorDataset:
    if isinstance(x, scipy.sparse.csr_matrix):
        x = x.toarray()
    x = torch.from_numpy(x)
    x = TensorDataset(x)
    
    return x

In [33]:
x_test = convert_text_to_transformer_input(tokenizer, df_test['words'].tolist())

In [20]:
unique_list = df_test['classes'].unique()

dic_count = dict()
count = 0

for el in unique_list:
    count += 1
    dic_count.update({el:count}) # create dict mapping for unique classes
    

In [29]:
df_test['classes_int'] = df_test['classes'].apply(lambda x: dic_count.get(x))

In [34]:
y_test = np_array_to_tensor_dataset(df_test['classes_int'].values)

In [35]:
joblib.dump(x_test, './data/test/x_test.lib')
joblib.dump(y_test, './data/test/y_test.lib')

['./data/test/y_test.lib']

In [39]:
notes_1grams = pd.read_csv('./data/train/ngrams/notes_1grams.csv', low_memory=False, chunksize=500000, index_col = False)

In [40]:
ls_matrix = []

for chunk in notes_1grams:
    chunk = chunk.dropna()
    temp_matrix = convert_text_to_transformer_input(tokenizer, chunk['ngrams'].tolist())
    ls_matrix.append(temp_matrix)

In [38]:
x_train = torch.utils.data.ConcatDataset(ls_matrix)

In [ ]:
joblib.dump(x_train, './data/train/x_train.lib')

Now that we have created all our necessary matrices from the original data, we will put all of it into knodle to build our models. For the modeling part please refer to the notebook "data_modeling".